In [2]:
# Karly July 11

# Phase 2 - stock direction
# Can you use eight years of daily news headlines to predict stock market movement?
# You have historical news headlines from Reddit WorldNews Channel ranked by reddit users
# votes, and only the top 25 headlines are considered for a single date. (Range: 2008-06-08 to 2016-07-01). 
# You are also provided with Dow Jones Industrial Average (DJIA) stock data (Range: 2008-08-08 to 2016-07-01). 
# The challenge is to use news headlines data to predict stock directionality, i.e. whether the DJIA Adj. 
# Close value decreases, or stays the same/increases.

In [3]:
import pandas as pd
import numpy as np
#import nltk
#import tensorflow
from nltk.metrics import edit_distance  

In [4]:
# The "train.csv" = your training dataset
with open('train.csv') as f:
    trainData = pd.read_csv(f, index_col=0, encoding='utf-8')
    
#with open('Kaggletestfile.csv') as f:
#    Data1 = pd.read_csv(f, index_col=0, encoding='utf-8')
    
# The "test.csv" = your test dataset used to grade your results
with open('test.csv') as f:
    testData = pd.read_csv(f, index_col=0, encoding='utf-8')

In [5]:
trainData.head()

,Open,High,Low,Close,Volume,Adj Close,Top1,Top2,Top3,Top4,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
8/26/15,15676.25977,16303.75000,15676.25977,16285.50977,208420000,16285.50977,Switzerland has completed the construction of ...,Ancient Greek palace unearthed near Sparta dat...,Nearly a billion monarch butterflies have vani...,Secret 'dark net' operation saves scores of ch...,...,Snowden of the Sea: Researchers Name New Crayf...,Canada has engineered bacteria that send a rem...,Migrants crisis: Germany's Merkel booed at Hei...,Thailand destroys ivory stockpile in ceremony ...,A pioneering procedure has led to a baby being...,Trans-Pacific Trade Agreement: Potential damag...,Germany has reportedly begun taking steps towa...,Senior Iran aide: Israel should be annihilated...,Italy hits back at Merkel in migrants row - It...,Court orders Russia to pay compensation for Gr...
1/12/12,12449.91016,12483.70020,12385.08008,12471.01953,128230000,12471.01953,Loggers in Brazil captured an eight-year-old g...,300 Chinese Foxconn Workers 'Threaten Mass Sui...,Tens of Thousands living in 6x2ft rabbit cages...,"Mexicos drug war kills more than 47,000 in 5 y...",...,Haiti: 2 Years After the Quake [pics],Canadian military containers from Afghanistan ...,Brazil says no evidence loggers burned indigen...,Afghanistans President Karzai condemns video s...,Iran's currency in freefall --- Iran's currenc...,Iran Says Scientists Murder Reveals Global Ter...,UK's Duchess of York charged in Turkey with il...,Opium price rises by 133% in Afghanistan,Holland bans khat leaves: Chewing of khat can ...,Why Are England and Scotland Splitting? - \nTo...
11/2/12,13232.62012,13289.45020,13076.57031,13093.16016,137660000,13093.16016,The home front: Man builds 60ft-long World War...,Pakistani couple kill daughter for talking to ...,"Thousands of Iranians chanting ""Death to Ameri...",Pakistani coupe kill their daughter by pouring...,...,Return of the Czech communists: the country's ...,"Sean Quinn, once Ireland's richest man, is giv...",Greek court dismisses charges against 'Lagarde...,"Greek journalist who published names of 2,000...","In Brazil, researchers genetically modifying m...",Iskandar Malaysia: the green mega-city rising ...,Japan protests after new report of incident in...,U.S. quietly eases rules on sale of medicines ...,"Hit by crisis, Greek society in free-fall",Is this the darkest day for Russia under Putin...
8/8/08,11432.08984,11759.95996,11388.04004,11734.32031,212830000,11734.32031,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
12/31/14,17987.66016,18043.22070,17820.88086,17823.07031,82840000,17823.07031,AirAsia flight found at the bottom of the Java...,North Korean defector details 'human experimen...,Korean Air ex-executive Cho Hyun-ah arrested -...,"South Korean to drop Sony film ""The Interview""...",...,Newly published NSA documents show agency coul...,Britain says it cannot support new Palestinian...,Bolivias Morales: US 'Backs Drug Trafficking',Euro zone politicians are not obliged to rescu...,Greece Dissolves Parliament to Prepare for Ear...,German Chancellor Angela Merkel in a New Year'...,Al Shabaab leader killed in U.S. air strike in...,New solar power material converts 90 percent o...,China launches website laying claim to Senkaku...,At Least 33 Killed in Yemen Suic

In [6]:
testData.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
2/19/10,b'What a surprise...',"b'""The whole idea of communism is dead in Nort...","b""Selectively applying and breaking the law in...","b""BBC News - Dutch government 'collapses' in r...",b'Japan gets the nod to kill whales again',"b""Britain 'knew Mossad was using fake passport...",b'Lost Nazi nuke-project uranium found in Dutc...,"b""Assisted suicide: law to be decriminalised '...",b'BBC News - Incredible Footage of Madeira Flo...,"b'""I wanted to throw a wedding feast for you. ...",...,b'Scary Shit from Hungary',b'Britain denies any advance warning of plan t...,b'When China Rules the World',"b""Michel Chossudovsky, who's from an independe...",b'Dutch government collapses Amid disagreement...,"b""Iran's science progress fastest in world: Ca...",b'Bishop Walter Mixa faces continuing criticis...,b'St. Trinians and beyond -- Ronald Searle at 90',b'Britain at risk of worse deficit crisis than...,"b""Zahra's Paradise. A 7-languages webcomic on ..."
12/17/10,Richard Stallman: The actions against MasterCa...,The French Government Can Now Censor the Inter...,Julian Assange interview with BBC hours after ...,"""Now WikiLeaks has laid bare the lies and coll...",Google's Satellite Images Reveal Chinese 'Ghos...,WikiLeaks breaks no Australian laws: Australia...,The Lebanese army has dismantled two Israeli s...,"Mexicos Drug War Leaves 30,196 Dead in Four Years",WikiLeaks: Cables reveal U.S.-Cuban cooperatio...,WikiLeaks cables: Sudanese president 'stashed ...,...,A new Cold War is starting in Asia as Japan is...,Austrian man convicted for yodelling while Mus...,WikiLeaks army comes out to defend freedom of ...,Women with seditious eyes must cover up,Scores die as drones renew attack on Pakistan'...,CIA top spy flees Pakistan over lawsuit - The ...,Eamonn McCann: Why the US is up in arms over W...,South African politicians beaten by Israeli so...,Japan is changing its defence policy in respon...,More warnings from North Korea for retaliation...
10/26/15,College applications in UK to become anonymous...,Saudi Arabia to run out of cash in fewer than ...,Saudi Prince Arrested in Beirut on Drug Smuggling,Strong earthquake jolts Islamabad,UK accused of doing China's bidding after poli...,WHO: Processed meats cause cancer.,Dutch court revokes Scientology's tax-exempt s...,A Chinese biotechnological company announced t...,Activists urge Trudeau to defend Canadas copyr...,Poland has returned their government to the ri...,...,Scientifically validated anti-diabetes herbal ...,Muslim leader condemns beheading of South Suda...,Red meat cancer risk to be revealed by WHO,Pakistan uses Taliban to counter India: CIA ch...,Tour boat carrying 27 passengers sinks off Tof...,China to supply fuel to Nepal after protesters...,Europe is starting to shut down the free flow ...,Nine killed in gun battle between Turkish poli...,"Guatemala's jailed former president, Otto Pere...",Plane's Undercarriage Collapses During Landing
4/3/13,North Korea via Instagram,"""North Koreans are not getting any information...","Saudi Arabia will now let women ride bicycles,...",Brazilian men gangrape an American girl in a p...,Russian billionaire plans to make humans immor...,Indian woman burns her rapist to death,"""Gate to hell"" discovered by Italian scientists.",Sony makes more money selling life insurance t...,"China deploys tanks, armor to NK border",It's Believed China is Dramatically Under-Repo...,...,City plans to fight drug addiction with... mar...,New face-sized tarantula discovered in Sri Lan...,Bangladesh police have arrested three atheist ...,Germany Doesn't Get Much Sun. How Did It Becom...,Saudi court sentences man to be paralyzed,"""The Anonymous hacktivist collective announced...",Double standard emerges in Brazil van gang rap...,Slain dictator Gaddafis only daughter booted f...,Cyprus President's Family Transferred Tens O

In [7]:
# Add stock movement column to train: 1 if (DJIA Adj. Close - Open)>= 0 | 0 otherwise)
trainData['stockMovement'] = np.where(trainData['Adj Close'] - trainData['Open'] >= 0, 1, 0)

In [8]:
trainData

,Open,High,Low,Close,Volume,Adj Close,Top1,Top2,Top3,Top4,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,stockMovement
Date,,,,,,,,,,,,,,,,,,,,,
8/26/15,15676.259770,16303.750000,15676.259770,16285.509770,208420000,16285.509770,Switzerland has completed the construction of ...,Ancient Greek palace unearthed near Sparta dat...,Nearly a billion monarch butterflies have vani...,Secret 'dark net' operation saves scores of ch...,...,Canada has engineered bacteria that send a rem...,Migrants crisis: Germany's Merkel booed at Hei...,Thailand destroys ivory stockpile in ceremony ...,A pioneering procedure has led to a baby being...,Trans-Pacific Trade Agreement: Potential damag...,Germany has reportedly begun taking steps towa...,Senior Iran aide: Israel should be annihilated...,Italy hits back at Merkel in migrants row - It...,Court orders Russia to pay compensation for Gr...,1
1/12/12,12449.910160,12483.700200,12385.080080,12471.019530,128230000,12471.019530,Loggers in Brazil captured an eight-year-old g...,300 Chinese Foxconn Workers 'Threaten Mass Sui...,Tens of Thousands living in 6x2ft rabbit cages...,"Mexicos drug war kills more than 47,000 in 5 y...",...,Canadian military containers from Afghanistan ...,Brazil says no evidence loggers burned indigen...,Afghanistans President Karzai condemns video s...,Iran's currency in freefall --- Iran's currenc...,Iran Says Scientists Murder Reveals Global Ter...,UK's Duchess of York charged in Turkey with il...,Opium price rises by 133% in Afghanistan,Holland bans khat leaves: Chewing of khat can ...,Why Are England and Scotland Splitting? - \nTo...,1
11/2/12,13232.620120,13289.450200,13076.570310,13093.160160,137660000,13093.160160,The home front: Man builds 60ft-long World War...,Pakistani couple kill daughter for talking to ...,"Thousands of Iranians chanting ""Death to Ameri...",Pakistani coupe kill their daughter by pouring...,...,"Sean Quinn, once Ireland's richest man, is giv...",Greek court dismisses charges against 'Lagarde...,"Greek journalist who published names of 2,000...","In Brazil, researchers genetically modifying m...",Iskandar Malaysia: the green mega-city rising ...,Japan protests after new report of incident in...,U.S. quietly eases rules on sale of medicines ...,"Hit by crisis, Greek society in free-fall",Is this the darkest day for Russia under Putin...,0
8/8/08,11432.089840,11759.959960,11388.040040,11734.320310,212830000,11734.320310,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",1
12/31/14,17987.660160,18043.220700,17820.880860,17823.070310,82840000,17823.070310,AirAsia flight found at the bottom of the Java...,North Korean defector details 'human experimen...,Korean Air ex-executive Cho Hyun-ah arrested -...,"South Korean to drop Sony film ""The Interview""...",...,Britain says it cannot support new Palestinian...,Bolivias Morales: US 'Backs Drug Trafficking',Euro zone politicians are not obliged to rescu...,Greece Dissolves Parliament to Prepare for Ear...,German Chancellor Angela Merkel in a New Year'...,Al Shabaab leader killed in U.S. air strike in...,New solar power material converts 90 percent o...,China launches website laying claim to Senkaku...,At Least 33 Killed in Yemen Suicide Bombing,0
8/29/11,11286.650390,11541.780270,11286.580080,11539.250000,177540000,11539.250000,"Luxury, horror lurk in Gadhafi family compound",Disney's best-selling Cars toys are being made...,C

In [9]:
testData

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
2/19/10,b'What a surprise...',"b'""The whole idea of communism is dead in Nort...","b""Selectively applying and breaking the law in...","b""BBC News - Dutch government 'collapses' in r...",b'Japan gets the nod to kill whales again',"b""Britain 'knew Mossad was using fake passport...",b'Lost Nazi nuke-project uranium found in Dutc...,"b""Assisted suicide: law to be decriminalised '...",b'BBC News - Incredible Footage of Madeira Flo...,"b'""I wanted to throw a wedding feast for you. ...",...,b'Scary Shit from Hungary',b'Britain denies any advance warning of plan t...,b'When China Rules the World',"b""Michel Chossudovsky, who's from an independe...",b'Dutch government collapses Amid disagreement...,"b""Iran's science progress fastest in world: Ca...",b'Bishop Walter Mixa faces continuing criticis...,b'St. Trinians and beyond -- Ronald Searle at 90',b'Britain at risk of worse deficit crisis than...,"b""Zahra's Paradise. A 7-languages webcomic on ..."
12/17/10,Richard Stallman: The actions against MasterCa...,The French Government Can Now Censor the Inter...,Julian Assange interview with BBC hours after ...,"""Now WikiLeaks has laid bare the lies and coll...",Google's Satellite Images Reveal Chinese 'Ghos...,WikiLeaks breaks no Australian laws: Australia...,The Lebanese army has dismantled two Israeli s...,"Mexicos Drug War Leaves 30,196 Dead in Four Years",WikiLeaks: Cables reveal U.S.-Cuban cooperatio...,WikiLeaks cables: Sudanese president 'stashed ...,...,A new Cold War is starting in Asia as Japan is...,Austrian man convicted for yodelling while Mus...,WikiLeaks army comes out to defend freedom of ...,Women with seditious eyes must cover up,Scores die as drones renew attack on Pakistan'...,CIA top spy flees Pakistan over lawsuit - The ...,Eamonn McCann: Why the US is up in arms over W...,South African politicians beaten by Israeli so...,Japan is changing its defence policy in respon...,More warnings from North Korea for retaliation...
10/26/15,College applications in UK to become anonymous...,Saudi Arabia to run out of cash in fewer than ...,Saudi Prince Arrested in Beirut on Drug Smuggling,Strong earthquake jolts Islamabad,UK accused of doing China's bidding after poli...,WHO: Processed meats cause cancer.,Dutch court revokes Scientology's tax-exempt s...,A Chinese biotechnological company announced t...,Activists urge Trudeau to defend Canadas copyr...,Poland has returned their government to the ri...,...,Scientifically validated anti-diabetes herbal ...,Muslim leader condemns beheading of South Suda...,Red meat cancer risk to be revealed by WHO,Pakistan uses Taliban to counter India: CIA ch...,Tour boat carrying 27 passengers sinks off Tof...,China to supply fuel to Nepal after protesters...,Europe is starting to shut down the free flow ...,Nine killed in gun battle between Turkish poli...,"Guatemala's jailed former president, Otto Pere...",Plane's Undercarriage Collapses During Landing
4/3/13,North Korea via Instagram,"""North Koreans are not getting any information...","Saudi Arabia will now let women ride bicycles,...",Brazilian men gangrape an American girl in a p...,Russian billionaire plans to make humans immor...,Indian woman burns her rapist to death,"""Gate to hell"" discovered by Italian scientists.",Sony makes more money selling life insurance t...,"China deploys tanks, armor to NK border",It's Believed China is Dramatically Under-Repo...,...,City plans to fight drug addiction with... mar...,New face-sized tarantula discovered in Sri Lan...,Bangladesh police have arrested three atheist ...,Germany Doesn't Get Much Sun. How Did It Becom...,Saudi court sentences man to be paralyzed,"""The Anonymous hacktivist collective announced...",Double standard emerges in Brazil van gang rap...,Slain dictator Gaddafis only daughter booted f...,Cyprus President's Family Transferred Tens O